In [1]:
from feature import *
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import torch.nn.functional as F
from feature import *

In [2]:
# df = pd.read_excel('附件三（测试集）.xlsx')
# df[:2]
df = pd.read_csv('p4_problem.csv')
df[:2]

,mean_flux,std_flux,max_flux,min_flux,range_flux,skewness,symmetric_messure,max_freq,mean_mag,std_mag,...,slope_abs_mean,duty_cicle,rise_time,fall_time,increase_points,decrease_points,material,temperature,frequency,wave_label
0,2.929709e-12,0.044232,0.062438,-0.062771,0.125209,-0.005737,0.079563,55.000000,0.063037,1.414010,...,0.000244,50.097656,11.0,-75.0,0.0,0.0,0,25,56320,2
1,-3.222655e-11,0.044172,0.062242,-0.062587,0.124829,-0.005754,0.079495,77.597656,0.062956,1.412101,...,0.000243,50.097656,18.0,-74.0,0.0,0.0,0,25,79460,2


In [15]:
# signal = df.iloc[0,5:].astype(float)
# fs = df.iloc[0,2].astype(float)
# signal

(0（磁通密度B，T）    0.002216
 1             0.002597
 2             0.002977
 3             0.003358
 4             0.003739
                 ...   
 1019          0.000308
 1020          0.000689
 1021          0.001070
 1022          0.001452
 1023          0.001834
 Name: 0, Length: 1024, dtype: float64,
 np.float64(56320.0))

In [5]:
def extract_features(row):
    flux_density = row.iloc[4:]  # 假设磁通密度列从第5列开始
    # print(type(flux_arr))
    flux_arr = flux_density.values.astype(float)
    print(len(flux_arr))
    fs = float( row.iloc[1] )
    print(f"fs{fs} type {type(fs)}")
    skewness, symmetric_messure = calculate_symmetry(flux_arr)
    max_freq, mean_mag, std_mag, energy = extract_frequency_features(flux_arr, fs)
    rise_time, fall_time = calculate_rise_and_fall(flux_arr)
    increase_points, decrease_points = calculate_sudden(flux_arr)
    slope = calculate_slope(flux_arr)
    return pd.Series({
        'mean_flux': np.mean(flux_arr),
        'std_flux': np.std(flux_arr),
        'max_flux': np.max(flux_arr),
        'min_flux': np.min(flux_arr),
        'range_flux': np.max(flux_arr) - np.min(flux_arr),
        'skewness': skewness,
        'symmetric_messure': symmetric_messure,
        'max_freq': max_freq,
        'mean_mag': mean_mag,
        'std_mag': std_mag,
        'energy': energy,
        'slope_mean': np.mean(slope),
        'slope_std': np.std(slope),
        'slope_max': np.max(slope),
        'slope_min': np.min(slope),
        'slope_abs_mean': np.mean(np.abs(slope)),
        'duty_cicle': calculate_duty_cycle(flux_arr),
        'rise_time': rise_time,
        'fall_time': fall_time,
        'increase_points':increase_points,
        'decrease_points':decrease_points,
    })

# features = merged_data.apply(extract_features, axis=1)

In [23]:
haha = pd.read_csv('all_train_full_loss.csv')
haha[:2]

,temperature,frequency,core_loss,waveform,0,1,2,3,4,5,...,1015,1016,1017,1018,1019,1020,1021,1022,1023,material
0,25,50030,1997.95525,正弦波,0.000213,0.000389,0.000566,0.000743,0.000919,0.001096,...,-0.001374,-0.001198,-0.001022,-0.000846,-0.000669,-0.000492,-0.000316,-0.000140,0.000036,材料1
1,25,50020,2427.74983,正弦波,-0.000551,-0.000358,-0.000165,0.000028,0.000221,0.000413,...,-0.002284,-0.002091,-0.001899,-0.001707,-0.001515,-0.001322,-0.001129,-0.000937,-0.000744,材料1


In [24]:
a = encoder.fit_transform(haha['waveform'])
encoder.classes_

array(['三角波', '梯形波', '正弦波'], dtype=object)

In [14]:

material_label = encoder.fit_transform(df['磁芯材料'])
print(encoder.classes_)
wave_label = encoder.fit_transform( df['励磁波形'])
encoder.classes_

['材料1' '材料2' '材料3' '材料4']


array(['三角波', '梯形波', '正弦波'], dtype=object)

In [20]:
encoder.classes_

array(['三角波', '梯形波', '正弦波'], dtype=object)

In [15]:
df_features['material'] = material_label
df_features['temperature'] = df['温度，oC']
df_features['frequency'] = df['频率，Hz']
df_features['wave_label'] = wave_label
df_features[:2]

,mean_flux,std_flux,max_flux,min_flux,range_flux,skewness,symmetric_messure,max_freq,mean_mag,std_mag,...,slope_abs_mean,duty_cicle,rise_time,fall_time,increase_points,decrease_points,material,temperature,frequency,wave_label
0,2.929709e-12,0.044232,0.062438,-0.062771,0.125209,-0.005737,0.079563,55.000000,0.063037,1.414010,...,0.000244,50.097656,11.0,-75.0,0.0,0.0,0,25,56320,2
1,-3.222655e-11,0.044172,0.062242,-0.062587,0.124829,-0.005754,0.079495,77.597656,0.062956,1.412101,...,0.000243,50.097656,18.0,-74.0,0.0,0.0,0,25,79460,2


In [18]:
df_features.shape
df_features.to_csv('p4_problem.csv', encoding='utf-8', index=False)

In [16]:
len(df_features)

400

In [23]:
df_features.to_csv('features25_and_loss.csv', index=False, encoding='utf-8-sig')

In [32]:
df_features_normalized.to_csv('features25_and_loss_normalized.csv', index=False, encoding='utf-8-sig')

In [29]:
df_features_normalized = (df_features.iloc[:,:-1] - df_features.iloc[:,:-1].min()) / (df_features.iloc[:,:-1].max() - df_features.iloc[:,:-1].min()) * 2 -1

In [31]:
df_features_normalized['core_loss'] = df_features['core_loss']
df_features_normalized[:2]

,mean_flux,std_flux,max_flux,min_flux,range_flux,skewness,symmetric_messure,max_freq,mean_mag,std_mag,...,duty_cicle,rise_time,fall_time,increase_points,decrease_points,material,temperature,frequency,wave_label,core_loss
0,-0.000124,-0.857452,-0.873468,0.871463,-0.872738,-0.249633,0.041179,-0.999125,-0.929511,-0.857257,...,0.142857,-0.941284,-0.25,-1.0,-1.0,-1.0,-1.0,-0.999823,1.0,1998.0
1,-0.000079,-0.839898,-0.856537,0.854389,-0.855733,-0.260879,0.051104,-0.998887,-0.919334,-0.839699,...,0.285714,-0.926606,-0.25,-1.0,-1.0,-1.0,-1.0,-0.999867,1.0,2428.0


In [7]:

X = torch.tensor(df.values, dtype=torch.float32)
df.values

array([[ 2.92970946e-12,  4.42317090e-02,  6.24378860e-02, ...,
         2.50000000e+01,  5.63200000e+04,  2.00000000e+00],
       [-3.22265500e-11,  4.41720048e-02,  6.22418210e-02, ...,
         2.50000000e+01,  7.94600000e+04,  2.00000000e+00],
       [-4.00390367e-11,  4.97226695e-02,  7.00207870e-02, ...,
         2.50000000e+01,  8.90800000e+04,  2.00000000e+00],
       ...,
       [-6.83598497e-12,  8.37918956e-02,  1.37639501e-01, ...,
         9.00000000e+01,  1.58750000e+05,  1.00000000e+00],
       [ 1.95319520e-12,  8.34761249e-02,  1.37186106e-01, ...,
         9.00000000e+01,  1.77900000e+05,  1.00000000e+00],
       [-5.37109387e-11,  6.64378163e-03,  1.09624870e-02, ...,
         9.00000000e+01,  1.99190000e+05,  1.00000000e+00]])

In [8]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled
X_scaled.shape

(400, 25)

In [27]:
df[:2]

,mean_flux,std_flux,max_flux,min_flux,range_flux,skewness,symmetric_messure,max_freq,mean_mag,std_mag,...,slope_abs_mean,duty_cicle,rise_time,fall_time,increase_points,decrease_points,material,temperature,frequency,wave_label
0,2.929709e-12,0.044232,0.062438,-0.062771,0.125209,-0.005737,0.079563,55.000000,0.063037,1.414010,...,0.000244,50.097656,11.0,-75.0,0.0,0.0,0,25,56320,2
1,-3.222655e-11,0.044172,0.062242,-0.062587,0.124829,-0.005754,0.079495,77.597656,0.062956,1.412101,...,0.000243,50.097656,18.0,-74.0,0.0,0.0,0,25,79460,2


In [3]:
# Define the neural network
class NeuralNet(nn.Module):
    def __init__(self, input_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.nn.Softplus()( self.fc4(x)) + 100
        return x

In [9]:
input_size = X_scaled.shape[1]
model = NeuralNet(input_size)

# Load the saved weights into the model
model.load_state_dict(torch.load('p4_nn_weights.pth', weights_only=True))

# Set the model to evaluation mode before making predictions
model.eval()

NeuralNet(
  (fc1): Linear(in_features=25, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=1, bias=True)
)

In [10]:

X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
ret = model(X_tensor)
ret

tensor([[8.5783e+03],
        [1.2953e+04],
        [1.9960e+04],
        [1.7752e+04],
        [2.9831e+04],
        [9.5033e+03],
        [2.1309e+03],
        [1.7861e+03],
        [2.3374e+05],
        [6.2831e+03],
        [8.7441e+03],
        [7.0047e+04],
        [2.4592e+03],
        [1.9210e+03],
        [1.3581e+04],
        [1.5578e+03],
        [5.9794e+04],
        [1.4284e+05],
        [2.0978e+03],
        [2.8571e+05],
        [8.6954e+02],
        [1.7885e+04],
        [1.0867e+05],
        [4.4785e+03],
        [1.0598e+04],
        [8.6880e+03],
        [8.5755e+04],
        [1.8337e+05],
        [1.2545e+03],
        [2.5129e+05],
        [3.3852e+03],
        [1.8174e+03],
        [1.7844e+04],
        [2.8439e+04],
        [4.8531e+03],
        [6.1572e+03],
        [2.4038e+04],
        [8.9414e+04],
        [1.1639e+05],
        [3.3510e+05],
        [3.8140e+04],
        [3.9388e+05],
        [2.8676e+05],
        [4.1276e+05],
        [3.9612e+03],
        [2

In [26]:
arr = ret.detach().numpy()
rsp = np.round(arr.flatten().tolist(), 1).tolist()
rsp

[400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.6,
 400.7,
 400.7,
 400.6,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.6,
 400.7,
 400.6,
 400.7,
 400.6,
 400.6,
 400.7,
 400.7,
 400.7,
 400.7,
 400.6,
 400.7,
 400.6,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.6,
 400.6,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.6,
 400.6,
 400.6,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.6,
 400.7,
 400.7,
 400.7,
 400.7,
 400.6,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.6,
 400.7,
 400.7,
 400.7,
 400.7,
 400.6,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.6,
 400.7,
 400.6,
 400.7,
 400.6,
 400.6,
 400.7,
 400.6,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,
 400.7,


In [24]:
with open('p4_answer.txt', 'w') as f:
    f.write("\n".join([str(i) for i in rsp]))